In [1]:
# wavefunction.py
"""
Trains the Lattice-PsiFormer for a *single* (t,V) pair on synthetic targets,
then prints the first 10 complex coefficients ψ(n; t,V).

Swap OCC/TARGET with your exact-diagonalisation data for real physics runs.
"""
import jax, jax.numpy as jnp

In [2]:
# Example output, takes about 5 min to run
from train import (                     
    OCC,                    # (H, N_sites)  integer array of Fock states
    create_state,           # helper that initialises model & optimiser
    train_step,             # one SGD step (jit-compiled)
    model                   # the LatticePsiFormer instance
)

print("Detected JAX backend →", jax.default_backend().upper(),
      "with", len(jax.devices()), "device(s)")

# -------- training set: choose a (t,V) and build batch ----------------------
t, V = 0.5, 4.0
TV_BATCH = jnp.tile(jnp.array([t, V], dtype=jnp.float32), (OCC.shape[0], 1))

# synthetic target coefficients (Re, Im)  –– replace with ED data
key    = jax.random.PRNGKey(123)
TARGET = jax.random.normal(key, (OCC.shape[0], 2)) * 0.1

# -------- quick training ----------------------------------------------------
state = create_state(jax.random.PRNGKey(0))

for epoch in range(1200):                
    state, loss = train_step(state, OCC, TV_BATCH, TARGET)
    if epoch % 300 == 0:
        print(f"epoch {epoch:4d} | MSE = {loss:.3e}")

# -------- wave-function output ---------------------------------------------
coeffs = model.apply(state.params, OCC, TV_BATCH, train=False)  # (H, 2)
print("\nFirst 10 coefficients for (t,V)=(", t, ",", V, "):")
for i in range(10):
    n_str = ''.join(str(int(b)) for b in OCC[i])
    re, im = map(float, coeffs[i])
    print(f"|{n_str}⟩  →  {re:+.5f}  {im:+.5f} i")


Detected JAX backend → CPU with 1 device(s)
epoch    0 | MSE = 9.984e-01
epoch  300 | MSE = 7.962e-01
epoch  600 | MSE = 9.956e-01
epoch  900 | MSE = 8.566e-01

First 10 coefficients for (t,V)=( 0.5 , 4.0 ):
|1111100000⟩  →  +0.78708  +7.96918 i
|1111010000⟩  →  +1.64936  +4.33142 i
|1111001000⟩  →  -2.59221  -1.57305 i
|1111000100⟩  →  -11.15819  +3.05209 i
|1111000010⟩  →  -0.65483  -4.02020 i
|1111000001⟩  →  +3.58885  +4.49224 i
|1110110000⟩  →  -2.93802  -1.41375 i
|1110101000⟩  →  -3.81637  -0.89691 i
|1110100100⟩  →  -7.02246  +0.06303 i
|1110100010⟩  →  +3.34094  +2.98012 i
